Will define a new ABC to support user provided non-repeating random-picking classes. *Bingo cages* and *lottery blowers* would be examples of such classes, derived from ABC *Tombola*

The `Tombola` has four methods:

- `.load()` put items into the container (abstract)
- `.pick()` remove one item from the container, returning it (abstract)
- `.loaded()` return `True` if there is at least one item in the container.
- `.inspect()` return a sorted `tuple` built from the items currently in the container, without changing its contents.

In [1]:
import abc

class Tombola(abc.ABC):   # subclass from ABC
    
    @abc.abstractmethod
    def load(self, iterable):
        """Add items from an iterable."""
        
    @abc.abstractmethod
    def pick(self):
        """Remove item at random, returning it.
        
        Raises `Lookup Error` when instance is empty.
        """
        
    def loaded(self):
        """Return `True` if there is at least 1 item, `False` otherwise."""
        return bool(self.inspect())
    
    def inspect(self):
        """Return a sorted tuple with the items currently inside."""
        items = []
        # we do not know how concreate classes will store data
        # we must use only the interface defined by the ABC
        # interface ~ other concrete or abstract methods or properties of the ABC
        while True:
            try:
                items.append(self.pick())
            except LookupError:
                break
        self.load(items)
        return tuple(sorted(items))

The abstract methods in ABC may have actuall implementation. Still, subclasses will be forced to override them, but they will be able to invoke them with `super()`.

In [2]:
class Fake(Tombola):
    def pick(self):
        return 12

In [3]:
print(repr(Fake))

<class '__main__.Fake'>


In [4]:
Fake.__mro__

(__main__.Fake, __main__.Tombola, abc.ABC, object)

In [5]:
f = Fake()

TypeError: Can't instantiate abstract class Fake with abstract methods load

In [6]:
import random

class BingoCage(Tombola):
    
    def __init__(self, items):
        self._randomizer = random.SystemRandom()
        self._items = []
        self.load(items)
        
    def load(self, items):
        self._items.extend(items)
        self._randomizer.shuffle(self._items)
        
    def pick(self):
        try:
            return self._items.pop()
        except IndexError:
            raise LookupError('Pick from empty BingoCage')
            
    def __call__(self):
        self.pick()

In [7]:
bc = BingoCage([*range(1,21)])

In [8]:
bc.inspect()

(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20)

In [9]:
bc.pick()

3

In [10]:
bc.inspect()

(1, 2, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20)

In [11]:
bc()

In [12]:
bc.inspect()

(1, 2, 4, 5, 6, 7, 8, 9, 10, 11, 12, 14, 15, 16, 17, 18, 19, 20)

In [13]:
class LotteryBlower(Tombola):
    
    def __init__(self, iterable):
        self._balls = list(iterable)
        
    def load(self, iterable):
        self._balls.extend(iterable)
        
    def pick(self):
        try:
            position = random.randrange(len(self._balls))
        except ValueError:
            raise LookupError('pick from empty LotteryBlower')
        return self._balls.pop(position)
    
    def loaded(self):
        return bool(self._balls)
    
    def inspect(self):
        return tuple(sorted(self._balls))

`self._balls = list(iterable)` is an example of the important idiom, `_balls` store a list of items constructed from this iterable. This makes our class flexible as it can accept any type that is iterable, but at the same time, we can pop items from the underlying list. Even if there is list passed to it it is good practice to produce a copy of it to avoid changing the underlying data structure.

In [14]:
lb = LotteryBlower(range(1, 21))

In [15]:
lb.inspect()

(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20)

In [16]:
lb.pick()

11

In [17]:
lb.inspect()

(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 12, 13, 14, 15, 16, 17, 18, 19, 20)

## Virtual subclass of Tombola
Virtual subclasses do not inherit from the registered ABCs, and are not checked for conformance to the ABC interface at any time, not even when they are instantiated. It's up to the subclass to actually implement all the methods needed to avoid runtime errors.

In [18]:
@Tombola.register
class TomboList(list):
    
    def pick(self):
        if self:
            position = random.randrange(len(self))
            return self.pop(position)
        else:
            raise LookupError('pop from empty TomboList')
            
    load = list.extend
    
    def loaded(self):
        return bool(self)
    
    def inspect(self):
        return tuple(sorted(self))
        

In [19]:
issubclass(TomboList, Tombola)

True

In [20]:
t = TomboList(range(100))

In [21]:
isinstance(t, Tombola)

True

In [22]:
TomboList.__mro__

(__main__.TomboList, list, object)